Predict the closest block that will get the driver the highest tips based on the time of the day and day of the week.

In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
data_2012_01=pd.read_csv("/Users/RAFAELLICHEN/Downloads/column_deleted_data_2012/yellow_tripdata_2012-01.csv")
data_2012_01=data_2012_01.drop("Unnamed: 0",1)
data_2012_01=data_2012_01.drop("passenger_count",1)
data_2012_01=data_2012_01.drop("dropoff_datetime",1)
data_2012_01=data_2012_01.drop("trip_distance",1)
data_2012_01=data_2012_01.drop("dropoff_longitude",1)
data_2012_01=data_2012_01.drop("dropoff_latitude",1)
data_2012_01=data_2012_01.drop("total_amount",1)
data_2012_01.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount
0,2012-01-10 23:55:50,-73.994693,40.725031,1.00
1,2012-01-11 19:18:25,-73.987955,40.752947,0.00
2,2012-01-11 19:19:19,-73.783093,40.648550,10.06
3,2012-01-11 19:19:21,-73.967515,40.758454,1.00
4,2012-01-11 14:38:15,-74.011315,40.711449,0.00


In [3]:
jan_time=pd.to_datetime(pd.Series(data_2012_01["pickup_datetime"]))
jan_time_list=[]
for index in jan_time:
    jan_time_list.append(index.dayofweek)
data_2012_01["weekday"]=jan_time_list
data_2012_01.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday
0,2012-01-10 23:55:50,-73.994693,40.725031,1.00,1
1,2012-01-11 19:18:25,-73.987955,40.752947,0.00,2
2,2012-01-11 19:19:19,-73.783093,40.648550,10.06,2
3,2012-01-11 19:19:21,-73.967515,40.758454,1.00,2
4,2012-01-11 14:38:15,-74.011315,40.711449,0.00,2


In [4]:
jan_2012_mon=pd.DataFrame(data_2012_01)
jan_2012_mon=jan_2012_mon.loc[jan_2012_mon["weekday"]==0]
jan_2012_mon=jan_2012_mon.reset_index(drop=True)
print(jan_2012_mon.shape)

(2098157, 5)


In [5]:
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude>40]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude<41]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude>-74.5]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude<-72.5]
jan_2012_mon.shape

(2048831, 5)

In [17]:
# assume our location is the following longitude and latitude

In [6]:
jan_2012_mon.head(1)

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0


In [16]:
# then we search within 1 mile of our location
# 1 mile change is approximately 0.018 degree change for coordinates

In [7]:
in_range=jan_2012_mon.loc[jan_2012_mon["pickup_longitude"]>=-73.993335-0.018]
in_range=in_range.loc[in_range["pickup_longitude"]<=-73.993335+0.018]
in_range=in_range.loc[in_range["pickup_latitude"]<=40.727717+0.018]
in_range=in_range.loc[in_range["pickup_latitude"]>=40.727717-0.018]
in_range.shape

(560846, 5)

In [14]:
import matplotlib.path as mplPath

def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones("/Users/RAFAELLICHEN/Documents/CSC 59970/nyc-yellow-taxi-prediction/datasets/block-groups-polygons.geojson")
    i=0
    for line in parts:
        if (line["pickup_longitude"] and line["pickup_latitude"]):
            pickup_location  = geom.Point(proj(float(line["pickup_longitude"]), float(line["pickup_latitude"])))
            try:
                block_id_list.append(findBlock(pickup_location, index, zones))
            except AttributeError:
                drop_list.append(i)
        i+=1

drop_list=[]
block_id_list=[]

In [15]:
mapToZone(in_range.T.to_dict().values())

In [18]:
print((len(drop_list)+len(block_id_list))==in_range.shape[0])

True


In [20]:
in_range=in_range.drop(in_range.index[drop_list])
in_range["block_id"]=block_id_list
in_range=in_range.reset_index(drop=True)
in_range=in_range.loc[in_range["block_id"]!=-1]
in_range.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday,block_id
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0,9547
1,2012-01-09 17:12:42,-74.004338,40.724819,0.0,0,9596
2,2012-01-09 22:00:58,-74.004840,40.730261,0.0,0,9529
3,2012-01-09 22:00:40,-73.998459,40.729385,0.0,0,10027
4,2012-01-09 22:03:06,-73.998232,40.726148,0.0,0,10117


In [25]:
# in_range.to_csv("2012-01-0-id.csv")

In [24]:
# now we have all data for monday of january 2012, 1 mile within your current location

In [ ]:
# get percentage for each block 
# get percentage for the entire data 

# one block/entire area
# (one block tipping rate)/(overall tipping rate)

In [25]:
in_range=pd.read_csv("/Users/RAFAELLICHEN/Downloads/2012-01-0-id.csv")
in_range=in_range.drop("Unnamed: 0",1)
in_range.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday,block_id
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0,9547
1,2012-01-09 17:12:42,-74.004338,40.724819,0.0,0,9596
2,2012-01-09 22:00:58,-74.004840,40.730261,0.0,0,9529
3,2012-01-09 22:00:40,-73.998459,40.729385,0.0,0,10027
4,2012-01-09 22:03:06,-73.998232,40.726148,0.0,0,10117


In [26]:
in_range_time_list=list(in_range["pickup_datetime"])
in_range_hour=[]
for e in in_range_time_list:
    in_range_hour.append(pd.to_datetime(e).hour)
in_range["time"]=in_range_hour
in_range.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday,block_id,time
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0,9547,19
1,2012-01-09 17:12:42,-74.004338,40.724819,0.0,0,9596,17
2,2012-01-09 22:00:58,-74.004840,40.730261,0.0,0,9529,22
3,2012-01-09 22:00:40,-73.998459,40.729385,0.0,0,10027,22
4,2012-01-09 22:03:06,-73.998232,40.726148,0.0,0,10117,22


In [24]:
in_range.to_csv("/Users/RAFAELLICHEN/Downloads/2012-01-0-id-time.csv")

In [85]:
from sklearn.cluster import KMeans
plon=in_range['pickup_longitude'].values
plat=in_range['pickup_latitude'].values
coodinates=np.array([[plon[i],plat[i]] for i in range(len(lon))])
kmeans_n = KMeans(n_clusters=111,  n_init=1)
kmeans_n.fit(coodinates)
labels = kmeans_n.labels_
in_range["label"]=labels
in_range.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday,block_id,time,label
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0,9547,19,47
1,2012-01-09 17:12:42,-74.004338,40.724819,0.0,0,9596,17,42
2,2012-01-09 22:00:58,-74.004840,40.730261,0.0,0,9529,22,10
3,2012-01-09 22:00:40,-73.998459,40.729385,0.0,0,10027,22,2
4,2012-01-09 22:03:06,-73.998232,40.726148,0.0,0,10117,22,90


In [132]:
ls=in_range.groupby('label').size()
ls=np.array([[ls[i]] for i in range(len(ls))])
lc=kmeans_n.cluster_centers_
train_s=int(len(ls)*0.6)
test_s=int(len(ls)*0.4)
train_f=ls[:train_s]
train_r=lc[:train_s]
test_f=ls[test_s:]
test_r=lc[test_s:]

In [135]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline


def fit_model(X, y):
    model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                ('linear', LinearRegression(fit_intercept=False))])
    model.fit(X, y)
    return model

def score_model(model, X, y, Xv, yv):
    return tuple([model.score(X, y), model.score(Xv, yv)])

def fit_model_and_score(data, response, validation, val_response):
    model = fit_model(data, response)
    return score_model(model, data, response, validation, val_response)

print (fit_model_and_score(train_f, train_r,
                           test_f, test_r))

(0.085833127196655798, 0.073589085352915223)


In [137]:
X=label_size
y=label_center

model=Pipeline([('poly', PolynomialFeatures(degree=2)),
                ('linear', LinearRegression(fit_intercept=False))])
model.fit(X, y)
X_predict=(max(ls))
y_predict = model.predict(X_predict)
y_predict

/Users/RAFAELLICHEN/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[-73.98367553,  40.72468407]])